In [1]:
import duckdb

test_db_string = "/Users/patrick/dev/ucl/comp0158_mscproject/database/test.db"

In [ ]:
# CREATE A TABLE
con = duckdb.connect(database=':memory:')
duckdb.sql("\
    CREATE TABLE TEST (\
        ID VARCHAR,\
    )")
con.close()

In [6]:
# DESCRIBE
con = duckdb.connect(database=':memory:')
res = duckdb.sql("DESCRIBE TEST")
print(res)
con.close()

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ID          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘



In [14]:
con = duckdb.connect(database=':memory:')        

duckdb.sql("CREATE TABLE MY_UNIPROT_DATA AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/uniprot/uniprot_reduced_1000.dat')")

print(duckdb.sql("DESCRIBE MY_UNIPROT_DATA"))
con.close()

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ column0     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ column1     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ column2     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘



In [11]:
# load file into table into database file
con = duckdb.connect(database=test_db_string)           
con.execute("CREATE TABLE MY_UNIPROT_DATA AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/uniprot/uniprotkb-2759_78494531_reduced.dat', columns={'id' :'VARCHAR', 'start': 'USMALLINT', 'end': 'USMALLINT'})")
description = con.execute("DESCRIBE MY_UNIPROT_DATA").fetchall()
print(description)
con.close()

[('id', 'VARCHAR', 'YES', None, None, None), ('start', 'USMALLINT', 'YES', None, None, None), ('end', 'USMALLINT', 'YES', None, None, None)]


In [3]:
import duckdb
db_string = "/Users/patrick/dev/ucl/comp0158_mscproject/database/proteins.db"
con = duckdb.connect(database=db_string)           
pfam_count = con.execute("SELECT COUNT(*) FROM PFAM_TOKEN").fetchall()
print(pfam_count)
con.close()

[(298766058,)]


In [2]:
import duckdb
db_string = "/Users/patrick/dev/ucl/comp0158_mscproject/database/test.db"
con = duckdb.connect(database=db_string)           
pfam_count = con.execute("SELECT COUNT(*) FROM MY_UNIPROT_DATA").fetchall()
print(pfam_count)
con.close()

[(78494529,)]


In [9]:
import duckdb
db_string = "/Users/patrick/dev/ucl/comp0158_mscproject/database/test.db"
output      = "/Users/patrick/dev/ucl/comp0158_mscproject/data/uniprot/proteins_ordered.dat"
con = duckdb.connect(database=db_string)

output_file = open(output, "w")

proteins = con.execute("SELECT * FROM MY_UNIPROT_DATA ORDER BY ID ASC").fetchall()

for i in range (len(proteins)):
    print(proteins[i])
    #print(proteins[i][0], '|', proteins[i][1])
    line = "|".join([proteins[i][0], str(proteins[i][1]), str(proteins[i][2])]) +'\n'
    output_file.write(line)
output_file.close()
con.close()

In [9]:
con = duckdb.connect(database=test_db_string)           
con.execute("DROP TABLE MY_UNIPROT_DATA")
con.close()

In [4]:
import duckdb
db_string = "/Users/patrick/dev/ucl/comp0158_mscproject/database/test.db"
con = duckdb.connect(database=db_string)        

#res = con.execute("DESCRIBE PFAM_TOKEN").fetchall()

# SELECT FROM LIST OF IDS - REALLY SLOW
#list = ['A0A010R6E0', 'A0A010RP22']
#entries = con.execute("SELECT * FROM PFAM_TOKEN WHERE column0 IN (SELECT UNNEST(?))", [list]).fetchall()

# FAST
entries_1 = con.execute("SELECT * FROM PFAM_TOKEN WHERE UNIPROT_ID = (?)", ['A0A010PZU8']).fetchall()

#print(res)
print(entries_1)
con.close()

IOException: IO Error: Could not set lock on file "/Users/patrick/dev/ucl/comp0158_mscproject/database/test.db": Conflicting lock is held in /Users/patrick/miniconda3/envs/conda_ucl_base/bin/python3.12 (PID 9680) by user patrick. See also https://duckdb.org/docs/connect/concurrency

## Unlock

In [1]:
import duckdb
import os
db_path = "/Users/patrick/dev/ucl/comp0158_mscproject/database/proteins.db"

def is_locked():
    db_path = "/Users/patrick/dev/ucl/comp0158_mscproject/database/proteins.db"

    lock_file = f'{db_path}.lock'
    return os.path.exists(lock_file)


is_locked()

# this works from a command prompt
#fuser database/proteins.db

# then kill the id if there is one liset

False